In [1]:
from flask import Flask, render_template, request
from PIL import Image
import numpy as np
from keras.models import load_model
import tensorflow as tf
import json
from ibm_watson import AssistantV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import import_ipynb
from credentials import credentials

Using TensorFlow backend.
/home/joaquin/anaconda3/envs/Alineacion_CNN/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/joaquin/anaconda3/envs/Alineacion_CNN/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/joaquin/anaconda3/envs/Alineacion_CNN/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

importing Jupyter notebook from credentials.ipynb
hoANOVbAuxWLcnmQzyxYcy03U092FIce-0aiUtDYeNo2 https://gateway.watsonplatform.net/assistant/api ab8eed2c-95d0-4d36-8b8e-50f9ac99167e


In [2]:
app = Flask(__name__, template_folder='templates')

def init():
    global model,graph
    # load the pre-trained Keras model
    model = load_model('model.h5')
    print(model.summary())
    graph = tf.get_default_graph()

@app.route('/')
def upload_file():
   return render_template('index.html')

In [3]:
@app.route('/uploader', methods = ['POST'])
def upload_image_file():
   if request.method == 'POST':
        img = Image.open(request.files['file'].stream).convert("L")
        img = img.resize((28,28))
        im2arr = np.array(img)
        im2arr = im2arr.reshape(1,28,28,1)
        with graph.as_default():
            y_pred = model.predict_classes(im2arr)
        
        predict = y_pred[0]
        watson(predict)

        return str(predict)

def watson(predict):
    
    apiKey, url, workspaceId, version = credentials()
    
    authenticator = IAMAuthenticator(apiKey)
    service = AssistantV1(
    version=version,
    authenticator = authenticator)
    
    service.set_service_url(url)
    
   
    response = service.message(
    workspace_id = workspaceId,
    input={
        'text': str(predict)
    }
    ).get_result()
    
    print(json.dumps(response, indent=2))

    print(predict)

In [ ]:
if __name__ == '__main__':
    print(("* Loading Keras model and Flask starting server..."
        "please wait until server has fully started"))
    init()
    app.run(debug = False)


* Loading Keras model and Flask starting server...please wait until server has fully started
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Feb/2020 15:52:44] "GET / HTTP/1.1" 200 -
